# Split-Delivery Vehicle Routing Problem (Algorithm)

## Data

In [1]:
!rm -rf ds/*
!cp -r datasets/SetAInstance001.orders ds/
!cp -r datasets/SetAInstance001.network ds/

In [2]:
ORDER_DATA_FILE = 'ds/SetAInstance001.orders'
NETWORK_DATA_FILE = 'ds/SetAInstance001.network'

In [3]:
# Restructure order data
with open(ORDER_DATA_FILE, 'r') as fin:
    data = fin.read().splitlines(True)
    ds: list = list()
    for l in data:
        ds.append(l.replace('#', ''))

with open(ORDER_DATA_FILE, 'w') as fout:
    fout.writelines(ds[1:])
    
# Restructure network data
with open(NETWORK_DATA_FILE, 'r') as fin:
    data = fin.read().splitlines(True)
    ds: list = list()
    for l in data:
        ds.append(l.replace('#', ''))
with open(NETWORK_DATA_FILE, 'w') as fout:
    fout.writelines(ds[1:])
    
splitted_data: list = list()

with open(NETWORK_DATA_FILE, 'r') as fin:
    data = fin.read().splitlines(True)
    ds: list = list()
    reached_distance_set: bool = False
    
    for l in data:
        if 'network information' in l:
            reached_distance_set = True
            continue
        if reached_distance_set:
            splitted_data.append(l)
            
with open(NETWORK_DATA_FILE, 'w') as fout:
    fout.writelines(splitted_data)

In [4]:
import pandas as pd

In [5]:
order_data: pd.DataFrame = pd.read_csv(ORDER_DATA_FILE, sep=';', parse_dates=True)
network_data: pd.DataFrame = pd.read_csv(NETWORK_DATA_FILE, sep=';', parse_dates=True)

In [6]:
order_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   ORD-keyword       500 non-null    object 
 1   number            500 non-null    int64  
 2   originIndex       500 non-null    int64  
 3   originCode        500 non-null    object 
 4   destinationIndex  500 non-null    int64  
 5   destinationCode   500 non-null    object 
 6   arrivalDate       500 non-null    object 
 7   dueDate           500 non-null    object 
 8   Unnamed: 8        0 non-null      float64
dtypes: float64(1), int64(3), object(5)
memory usage: 35.3+ KB


In [7]:
order_data.tail(10)

,ORD-keyword,number,originIndex,originCode,destinationIndex,destinationCode,arrivalDate,dueDate,Unnamed: 8
490,ORD,490,10,Wuppertal,13,Neuss,10.05.2022,11.05.2022,NaN
491,ORD,491,10,Wuppertal,13,Neuss,10.05.2022,12.05.2022,NaN
492,ORD,492,10,Wuppertal,13,Neuss,10.05.2022,11.05.2022,NaN
493,ORD,493,10,Wuppertal,7,Oberhausen,10.05.2022,12.05.2022,NaN
494,ORD,494,10,Wuppertal,7,Oberhausen,10.05.2022,11.05.2022,NaN
495,ORD,495,10,Wuppertal,0,Osnabrück,10.05.2022,12.05.2022,NaN
496,ORD,496,10,Wuppertal,27,Recklinghausen,10.05.2022,12.05.2022,NaN
497,ORD,497,10,Wuppertal,9,Solingen,10.05.2022,12.05.2022,NaN
498,ORD,498,10,Wuppertal,14,Viersen,10.05.2022,12.05.2022,NaN
499,ORD,499,10,Wuppertal,36,Witten,10.05.2022,11.05.2022,NaN


In [8]:
network_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1764 entries, 0 to 1763
Data columns (total 6 columns):
 #   Column                                Non-Null Count  Dtype 
---  ------                                --------------  ----- 
 0   ODP-keyword(Origin-Destination-Pair)  1764 non-null   object
 1   originIndex                           1764 non-null   int64 
 2   originName                            1764 non-null   object
 3   destinationIndex                      1764 non-null   int64 
 4   destinationName                       1764 non-null   object
 5   distanceKM                            1764 non-null   int64 
dtypes: int64(3), object(3)
memory usage: 82.8+ KB


In [9]:
network_data.head(25)

,ODP-keyword(Origin-Destination-Pair),originIndex,originName,destinationIndex,destinationName,distanceKM
0,ODP,0,Osnabrück,0,Osnabrück,0
1,ODP,0,Osnabrück,1,Düsseldorf,182
2,ODP,0,Osnabrück,2,Duisburg,161
3,ODP,0,Osnabrück,3,Essen,144
4,ODP,0,Osnabrück,4,Krefeld,183
5,ODP,0,Osnabrück,5,Mönchengladbach,205
6,ODP,0,Osnabrück,6,Mülheim an der Ruhr,154
7,ODP,0,Osnabrück,7,Oberhausen,152
8,ODP,0,Osnabrück,8,Remscheid,169
9,ODP,0,Osnabrück,9,Solingen,175


## Inspecting the dataset

In [10]:
order_data['arrivalDate'].min()

'01.05.2022'

In [11]:
order_data['arrivalDate'].max()

'10.05.2022'

In [12]:
order_data['dueDate'].min()

'02.05.2022'

In [13]:
order_data['dueDate'].max()

'12.05.2022'

# Order Allocation Algorithm (OAA)

## Grading System 

A grading system allows to score all orders based on fixed criteria. Afterwards, for each due date, the orders can be batched up to 6 cars per transport vehicle. For the grading system to be efficient, the criteria must optimize the cost of transport routes by minimizing the length while maximizing the amount of delivered cars per transporter.

In [14]:
from datetime import datetime, timedelta

def calculate_datetimes(today: str) -> (datetime, datetime, datetime):
    # Calculate datetimes for today (t), tomorrow (t+1), TDAD (t+2)
    today_datetime = datetime.strptime(today, '%d.%m.%Y')
    tomorrow_datetime = today_datetime + timedelta(days=1)
    tdat_datetime = tomorrow_datetime + timedelta(days=1)
    
    return (
        today_datetime,
        tomorrow_datetime,
        tdat_datetime,
    )

def grade_orders(
    orders: pd.DataFrame = order_data,
    network: pd.DataFrame = network_data,
    today: str = None,
) -> (pd.DataFrame, pd.DataFrame, dict[int, int]):
    if not today:
        today = orders['arrivalDate'].min()
    
    # Datetimes for today (t), tomorrow (t+1), TDAD (t+2)
    today_datetime, tomorrow_datetime, tdat_datetime = calculate_datetimes(today)
    
    # Reset scores for any previous grading.
    # Initially, every orders receives a score of 100 points.
    orders['score'] = 1
    
    # For every order, divide the score S by the number of days D until due date.
    # If only D = 1 day remains until due date, the score S is redeclared with S/D = S/1, leaving the score as it is.
    # If D > 1 day remains until due date, the score S is redeclared with S / D, decreasing the score.
    for index, row in orders.iterrows():
        arrival_datetime = datetime.strptime(row['arrivalDate'], '%d.%m.%Y')
        due_datetime = datetime.strptime(row['dueDate'], '%d.%m.%Y')
        days_delta = (due_datetime - arrival_datetime).days
        # Adjust score based on days delta
        orders.at[index, 'score'] = row['score'] / days_delta
    
    """
    # For every order, multiply the score S with the distance from the center.
    for index, row in orders.iterrows():
        # Get origin and destination index from order
        origin_index = row['originIndex']
        destination_index = row['destinationIndex']
        # Get distance from the origin (center) to the destination (dealer)
        distances = network.query(f"originIndex == {origin_index} and destinationIndex == {destination_index}")
        assert len(distances) == 1
        distance: int = distances['distanceKM']
        # Adjust score based on distance from center to destination
        orders.at[index, 'score'] = row['score'] * distance
    """
    
    # Calculating the mean distance between locations
    # TODO: calculating the mean without the distances from the center to the dealers
    mean_distance: float = network['distanceKM'].mean()
    print('LOG:', 'Mean distance:', mean_distance)
    
    # Compare every two orders with each other and determine whether their distance is less than the mean distance.
    # If their distance is less than the mean, they have a stronger metric relationship compared to other pairs.
    # If so, increasing the score will make both of them rank higher in the grading, which increases their possibility of being shipped together.
    for index, row in network.iterrows():
        origin_index = row['originIndex']
        destination_index = row['destinationIndex']
        distance = row['distanceKM']
        
        N = 5
        for n in range(1, N + 1):
            if distance <= (mean_distance / n):
                # - increase scores for origin, destination
                X = orders.query(f"destinationIndex == {destination_index}")
                Y = orders.query(f"destinationIndex == {origin_index}")
                
                for S in [X, Y]:
                    for i, r in X.iterrows():
                        orders.at[i, 'score'] = r['score'] * n
            else:
                break
    
    batches: dict[int, int] = {}
    
    td: pd.DataFrame = orders.query(f"arrivalDate == '{today_datetime.strftime('%d.%m.%Y')}' and dueDate == '{tomorrow_datetime.strftime('%d.%m.%Y')}'")
    ttd: pd.DataFrame = orders.query(f"arrivalDate == '{today_datetime.strftime('%d.%m.%Y')}' and dueDate == '{tdat_datetime.strftime('%d.%m.%Y')}'")
    
    deliveries: pd.DataFrame = pd.concat([td, ttd])
    
    # TODO: sort deliveries by descending score
    
    # Create a mapping between a dealer ID and the number of ordered cars that can or need to be delivered today.
    for index, row in deliveries.iterrows():
        destination_index = row['destinationIndex']
        
        if destination_index not in batches.keys():
            batches[destination_index] = 1
        else:
            batches[destination_index] = batches[destination_index] + 1
        
        # Drop out row to enable data consistency.
        # The manipulated dataset should be saved after the algorithm was executed.
        orders.drop(axis=0, index=index)
    
    return (orders, network, batches)

orders, network, batches = grade_orders()

LOG: Mean distance: 88.43197278911565


In [15]:
batches

{21: 1,
 28: 2,
 31: 3,
 16: 1,
 2: 3,
 1: 5,
 40: 2,
 33: 1,
 37: 1,
 17: 5,
 5: 3,
 6: 1,
 7: 3,
 11: 2,
 9: 2,
 12: 1,
 22: 1,
 32: 3,
 3: 1,
 34: 1,
 35: 1,
 24: 3,
 30: 2,
 10: 2}

In [16]:
# Test for the sum of cars that are being delivered today to make sure that
# the number of outgoing cars matches the number of incoming provided ones.
S = 0
for k, v in batches.items():
    S = S + v
S

50

In [17]:
def allocate(batches: dict[int, int]) -> list[list[int]]:
    allocations: list[list[int]] = []
    
    # Iterate over batched orders to check whether the amount of vehicles exceeds > 6.
    for key, value in batches.items():
        if value >= 6:
            X = value
            while X >= 6:
                allocations.append([key for key in range(0, 6)])
                X = X - 6
            batches[key] = X
    
    # Group remaining orders to allocations. 
    for k_x in batches.keys():
        allocation: list[int] = []
        leftover: int = 6 - batches.get(k_x)
        
        for k_y in batches.keys():
            if k_y != k_x:
                vehicles: int = batches.get(k_y)
                if leftover == vehicles:
                    allocation.concat([k_x for i in range(0, batches.get(k_x))])
                    allocation.concat([k_y for i in range(0, batches.get(k_y))])
        
            
            

In [18]:
d = {}
type(d.keys())

dict_keys